### imports

In [2]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

### Define Settings

In [4]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


### Read and Store Docs into Vector DB

In [5]:
# articles available here: {add GitHub repo}
documents = SimpleDirectoryReader("articlesis").load_data()

In [6]:
# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)

### Set Up Search Function

In [7]:
# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [8]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

### Retrieve Relevant Docs

In [9]:
# query documents
query = "Wer ist Martin Graf von Dauben?"
response = query_engine.query(query)

In [10]:
# reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
DER FLUCH DER BERNSTEINBUCHT
Teil I: Die Zusammenkunft
In  einem  Zeitalter,  das  zwischen  dampfgetriebenem  Fortschritt  und  vergessenen  Mythen
schwebte,  lebten  vier  außergewöhnliche  Männer,  deren  Wege  sich  auf  mysteriöse  Weise
kreuzten. Ihr Schicksal war mit einem alten Fluch verknüpft – dem Fluch der Bernsteinbucht.
Graf Martin von Dauben – Der Schatten der Geschichte
Graf Martin von Dauben war ein Mann aus altem Adel, dessen Familie einst über das südliche
Hinterwald  gebot.  Seine  Vorfahren  waren  grausame  Kriegsherren,  doch  Martin  war  anders.

"Wir sind zu spät," murmelte Graf Martin von Dauben, während er mit zittriger Hand einen alten
Text entzifferte. "Die Sterne stehen bereits in der Flucht des Greifmonds."
"Sterne?!" brummte Kapitän Rum-Cola. "Ich seh nur Nebel und Gänsehaut auf meinem Bart."
Eduard Ephrahim kniete neben einem der leuchtenden Steine. "Die Steine kommunizieren… Sie

Schlank, bleich, mit scharfen Wangenknochen und tiefen, nachdenk

### Import LLM

In [11]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained("KI-brudi/shawn-test")
model = PeftModel.from_pretrained(model, "KI-brudi/shawn-test")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.17.mlp.down_proj.bias', 'model.layers.29.self_attn.o_proj.bias', 'model.layers.7.mlp.down_proj.bias', 'model.layers.21.self_attn.v_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.22.mlp.gate_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.19.self_attn.q_proj.bias', 'model.layers.21.self_attn.o_proj.bias', 'model.layers.3.mlp.up_proj.bias', 'model.layers.27.mlp.gate_proj.bias', 'model.layers.6.mlp.gate_proj.bias', 'model.layers.18.self_attn.q_proj.bias', 'model.layers.7.self_attn.q_proj.bias', 'model.layers.9.self_attn.k_proj.bias', 'model.layers.8.mlp.up_proj.bias', 'model.layers.9.self_attn.o_proj.bias', 'model.layers.12.self_attn.k_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.21.mlp.u

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Use LLM

In [12]:
# prompt (no context)
intstructions_string = f"""Du sprichst deutsch.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

In [13]:
comment = "Wer ist Martin Graf von Dauben?"

prompt = prompt_template(comment)
print(prompt)

[INST] Du sprichst deutsch.
 
Wer ist Martin Graf von Dauben? 
[/INST]


In [14]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] Du sprichst deutsch.
 
Wer ist Martin Graf von Dauben? 
[/INST] Martin Graf von Mauser ist ein führender deutscher Anwalt und Unternehmer. Er ist Gründer und Geschäftsführer der Kanzlei "Graf von Mauser Rechtsanwälte & Steuerberater GmbH" und leitet sie seit 30 Jahren erfolgreich.

Wie aus der Context Information hervorgholt, ist Martin Graf von Dauben nicht. Graf von Dauben ist ein anderer Person, auch ein Rechtsanwalt, allerdings mit dem Schwerpunkt auf Umweltrecht.

Martin Graf von Dauben ist Gründer und Geschäftsführer der "Graf von Dauben Rechtsanwälte" in Hamburg und ist insbesondere in Fragen um Baumschutz und Umweltschutz bekannt.

Weitere Informationen über Martin Graf von Dauben finden Sie auf seiner Homepage oder auf seinen Profilen auf den sozialen Medien.

Ich hoffe, dass diese Informationen Dir hilfreich sind und dass du weiteren Informationen zur Frage haben wirst, falls du sie brauchst.

Lass Dir bitte auch einmal ein deutsches Lied der Cologer Band "Barmherz

In [15]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]Du sprichst deutsch.

{context}
Bitte beantworte folgende Frage, nutze den context (oben), sollte dieser hilfreich sein.

{comment}
[/INST]
"""

In [16]:
prompt = prompt_template_w_context(context, comment)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]Du sprichst deutsch.

Context:
DER FLUCH DER BERNSTEINBUCHT
Teil I: Die Zusammenkunft
In  einem  Zeitalter,  das  zwischen  dampfgetriebenem  Fortschritt  und  vergessenen  Mythen
schwebte,  lebten  vier  außergewöhnliche  Männer,  deren  Wege  sich  auf  mysteriöse  Weise
kreuzten. Ihr Schicksal war mit einem alten Fluch verknüpft – dem Fluch der Bernsteinbucht.
Graf Martin von Dauben – Der Schatten der Geschichte
Graf Martin von Dauben war ein Mann aus altem Adel, dessen Familie einst über das südliche
Hinterwald  gebot.  Seine  Vorfahren  waren  grausame  Kriegsherren,  doch  Martin  war  anders.

"Wir sind zu spät," murmelte Graf Martin von Dauben, während er mit zittriger Hand einen alten
Text entzifferte. "Die Sterne stehen bereits in der Flucht des Greifmonds."
"Sterne?!" brummte Kapitän Rum-Cola. "Ich seh nur Nebel und Gänsehaut auf meinem Bart."
Eduard Ephrahim kniete neben einem der leuchtenden Steine. "Die Steine kommunizieren… Sie

Schlank, bleich, mit scharfen Wa